Var. 118	Fmt: N2	Col: 16850-16851	Name: firm_location (DN)
	11b. Where is your firm located?
	1=US/Canada	2=Africa	3=Asia	4=Europe	5=Latin America	

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
import re
import pickle as pk

import seaborn as sns
import numpy as np
import nltk.tokenize as tokenize

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.multiclass import OneVsOneClassifier
from sklearn import cross_validation, grid_search
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn import linear_model, decomposition
from sklearn.svm import LinearSVC
import nltk

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords, words

import matplotlib
matplotlib.style.use('ggplot')

In [43]:
df_all = pd.read_csv('corporate_culture_trunc.csv', encoding='latin-1') #reads csv into dataframe
df_all = df_all.dropna(subset=['Q1']) #drop row if no description (NaN in Q1)
df_all = df_all.dropna(subset=['firm_location']) #drop row if no description (NaN in firm_location)
culture_text=df_all['Q1'] #takes only the first column

####US & CANADA = 1, REST = 0
df_all.loc[df_all['firm_location']==2, 'firm_location'] = 0
df_all.loc[df_all['firm_location']==3, 'firm_location'] = 0
df_all.loc[df_all['firm_location']==4, 'firm_location'] = 0
df_all.loc[df_all['firm_location']==5, 'firm_location'] = 0
df_all2 = df_all[df_all['firm_location']==1][0:400]
df_all2 = df_all2[['Q1','firm_location']]
df_all0 = df_all[df_all['firm_location']==0]
df_all0 = df_all0[['Q1','firm_location']]
print(len(df_all2))
df_all2.append(df_all0)
print(len(df_all2))

x_train, x_test, y_train, y_test = cross_validation.train_test_split(df_all2['Q1'],df_all2['firm_location'],test_size=0.2)

print(len(df_all2.loc[df_all['firm_location']==1]))
print(len(df_all2))

400
400
400
400


In [37]:
1169.0/1406.0

0.8314366998577525

In [30]:
mypipeline=Pipeline([
  ('cvect', CountVectorizer(ngram_range=(1, 2), stop_words=nltk.corpus.stopwords.words('english'))),
  ('tfidf', TfidfTransformer()),
  ('rclassify', linear_model.RidgeClassifier())
])

mypipeline.fit(x_train,y_train)
print(mypipeline.score(x_test,y_test))

## EIGHTY PERCENT ACCURACY RATE... Worse than guessing

0.808510638298


In [32]:
cvect = CountVectorizer(ngram_range=(1, 1), stop_words=nltk.corpus.stopwords.words('english'))
Xvect = cvect.fit_transform(df_all['Q1'])
print(Xvect.shape)[1]
Xvect_names = cvect.get_feature_names()
tfidf = TfidfTransformer()
Xtfidf = tfidf.fit_transform(Xvect)

x_train, x_test, y_train, y_test = cross_validation.train_test_split(Xtfidf,df_all['firm_location'],test_size=0.2)

ridge_reg = linear_model.RidgeClassifier()
ridge_reg.fit(x_train, y_train)
print(ridge_reg.score(x_test,y_test))
xarray = ridge_reg.coef_

xlist=[]
for i in range(xarray.shape[1]):
    if xarray[0][i] != 0:
        xlist.append({'word':Xvect_names[i], 'beta':xarray[0][i], 'absbeta':abs(xarray[0][i])})

2326
0.812056737589


In [33]:
newlist = sorted(xlist, key=lambda k: k['beta'],reverse=True)
for news in newlist[0:20]:
    print(news['word'],news['beta'])

(u'mind', -1.0887657955512611)
(u'urgent', -1.0745562049606348)
(u'entrepreneurship', -0.99315003914925926)
(u'opaque', -0.99128160513573038)
(u'closed', -0.98865378458487074)
(u'bureaucracy', -0.98737068704330277)
(u'social', -0.97479043103391416)
(u'individualistic', -0.91726202453623806)
(u'action', -0.90633608306572466)
(u'progressing', -0.88731503003992584)
(u'services', -0.84170933629765943)
(u'moderate', -0.83340561583755435)
(u'proud', -0.82912533777798403)
(u'knowledge', -0.82887407087342047)
(u'successful', -0.81956685544904795)
(u'comitment', -0.78807196234609833)
(u'missing', -0.78782993097832898)
(u'collegiate', -0.78782982908160137)
(u'light', -0.78782531534747824)
(u'aguantar', -0.78781685070177843)
(u'collegian', -0.78779786587134826)
(u'ghghhg', -0.78772791660369945)
(u'nice', -0.77319591658910458)
(u'governance', -0.7559016436893109)
(u'sharing', -0.75467603347566181)
(u'sincerity', -0.75301387203203851)
(u'determined', -0.74671135882647965)
(u'diversity', -0.74306500

Top words that people who like their company culture used [in order of importance]:

open, customer, integrity, professional, collaborative, right, healthy, opportunity, fun, team, clients, respect, supportive, innovation, collegial, relaxed, play, teamwork, strong, innovative, honesty, excellence, friendly, ethical, collaboration, stable, cfo, empowering, resonant, collegiate, caring, christian, responsibility


Top words that people who didn't like their company culture used [in order of importance]:

lack, toxic, change, silos, unstructured, opaque, unstructed, autocratic, bad, political, dysfunctional, secretive, bureaucracy, chaos, verify, siloed, little, stressful, top, fear, oppressive, low, great, sales, dictatorial, non